In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
import csv
import optuna

In [2]:
# 셀에서 출력 정확도 설정.
%precision %.3f

'%.3f'

In [3]:
file_path = r'.\crawling_data\preprocessed_essay.csv'

# 자소서 리스트화
documents = []
with open(file_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        documents.append(row[0])

documents = documents[:77] + documents[-77:]

In [4]:
# 전문가 평점 3점 이상인 상위 175개는 label에 1, 나머지 하위는 label에 0 값 부여
# 분류 예측용 label
labels = []
for i in range(154):
    if i < 77:
        labels.append(1)
    else:
        labels.append(0)

In [5]:
# 불용어 리스트 가져오기
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

documents_corpus = []
for document in documents:
    documents_corpus.append(preprocess(document))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(documents_corpus, labels, test_size=0.3, random_state=123)

In [7]:
# Tagged Corpus 생성.
# 모델 학습에 필요한 형태로 변환.
my_tagged_corpus = [TaggedDocument(words=my_words, tags=[i]) for i, my_words in enumerate(X_train)]

In [9]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300, step=10)
    window = trial.suggest_int('window', 2, 10)
    min_count = trial.suggest_int('min_count', 1, 10)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    negative = trial.suggest_int('negative', 3, 15)
    dm = trial.suggest_categorical('dm', [0, 1])

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, negative=negative, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # Randomforest 하이퍼파라미터 튜닝
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 5, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])

    # Randomforest 모델 생성
    model = RandomForestClassifier(n_estimators=n_estimators,
                                    max_depth=max_depth,
                                    min_samples_split=min_samples_split,
                                    min_samples_leaf=min_samples_leaf,
                                    max_features=max_features,
                                    random_state=123)
    
    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-21 14:47:17,519] A new study created in memory with name: no-name-ff4117b7-bfee-41bf-8beb-7a66eae6b2d3
[I 2024-06-21 14:47:29,395] Trial 0 finished with value: 0.631578947368421 and parameters: {'vector_size': 280, 'window': 2, 'min_count': 7, 'epochs': 20, 'negative': 8, 'dm': 0, 'n_estimators': 408, 'max_depth': 16, 'min_samples_split': 3, 'min_samples_leaf': 5, 'max_features': 'log2'}. Best is trial 0 with value: 0.631578947368421.
[I 2024-06-21 14:47:33,360] Trial 2 finished with value: 0.55 and parameters: {'vector_size': 250, 'window': 2, 'min_count': 6, 'epochs': 20, 'negative': 5, 'dm': 1, 'n_estimators': 388, 'max_depth': 30, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 'log2'}. Best is trial 0 with value: 0.631578947368421.
[I 2024-06-21 14:47:36,126] Trial 3 finished with value: 0.5306122448979592 and parameters: {'vector_size': 110, 'window': 6, 'min_count': 1, 'epochs': 35, 'negative': 9, 'dm': 0, 'n_estimators': 296, 'max_depth': 15, 'min_sam

Best hyperparameters:  {'vector_size': 180, 'window': 4, 'min_count': 9, 'epochs': 45, 'negative': 12, 'dm': 0, 'n_estimators': 243, 'max_depth': 19, 'min_samples_split': 7, 'min_samples_leaf': 3, 'max_features': 'sqrt'}


In [10]:
best_params = study.best_params

# Doc2Vec 모델 생성
model_rf = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], 
                negative=best_params['negative'], seed=123)

# RandomForest 모델 생성
clf_rf = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                            max_depth=best_params['max_depth'],
                            min_samples_split=best_params['min_samples_split'],
                            min_samples_leaf=best_params['min_samples_leaf'],
                            max_features=best_params['max_features'],
                            random_state=123)

# Doc2Vec 벡터 생성
X_train_vec = [model_rf.infer_vector(doc) for doc in X_train]
X_test_vec = [model_rf.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf_rf.fit(X_train_vec, y_train)
y_pred = clf_rf.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.80      0.50      0.62        24
           1       0.62      0.87      0.73        23

    accuracy                           0.68        47
   macro avg       0.71      0.68      0.67        47
weighted avg       0.71      0.68      0.67        47



In [11]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300, step=10)
    window = trial.suggest_int('window', 2, 10)
    min_count = trial.suggest_int('min_count', 1, 10)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    negative = trial.suggest_int('negative', 3, 15)
    dm = trial.suggest_categorical('dm', [0, 1])
    
    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, negative=negative, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # SVM 하이퍼파라미터 튜닝
    C = trial.suggest_float('C', 0.01, 100.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3
    gamma = trial.suggest_float('gamma', 1e-4, 1.0, log=True) if kernel in ['rbf', 'poly'] else 'scale'

    # SVM 모델 생성
    model = SVC(
        C=C,
        kernel=kernel,
        degree=degree,
        gamma=gamma,
        random_state=123
    )

    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-21 15:01:17,043] A new study created in memory with name: no-name-728ea3bf-ad34-444c-97ac-367477a086e4
[I 2024-06-21 15:01:29,580] Trial 0 finished with value: 0.6571428571428571 and parameters: {'vector_size': 120, 'window': 5, 'min_count': 4, 'epochs': 20, 'negative': 15, 'dm': 0, 'C': 1.3464443490431894, 'kernel': 'poly', 'degree': 4, 'gamma': 0.008485695313886038}. Best is trial 0 with value: 0.6571428571428571.
[I 2024-06-21 15:01:30,255] Trial 4 finished with value: 0.4375 and parameters: {'vector_size': 250, 'window': 3, 'min_count': 3, 'epochs': 25, 'negative': 7, 'dm': 0, 'C': 5.649236972219222, 'kernel': 'rbf', 'gamma': 0.0033752933717445485}. Best is trial 0 with value: 0.6571428571428571.
[I 2024-06-21 15:01:34,111] Trial 3 finished with value: 0.6571428571428571 and parameters: {'vector_size': 260, 'window': 4, 'min_count': 3, 'epochs': 40, 'negative': 5, 'dm': 0, 'C': 0.2795627842967626, 'kernel': 'poly', 'degree': 2, 'gamma': 0.0037592504574731982}. Best is tr

Best hyperparameters:  {'vector_size': 300, 'window': 4, 'min_count': 1, 'epochs': 25, 'negative': 6, 'dm': 1, 'C': 13.571012432732546, 'kernel': 'rbf', 'gamma': 0.07780639244535713}


In [12]:
best_params = study.best_params

# Doc2Vec 모델 생성
model_svc = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], 
                negative=best_params['negative'], seed=123)

# SVM 분류 모델 생성
clf_svc = SVC(C=best_params['C'], kernel=best_params['kernel'], gamma=best_params['gamma'], random_state=123)#degree=best_params['degree'], gamma=best_params['gamma'], 

# Doc2Vec 벡터 생성
X_train_vec = [model_svc.infer_vector(doc) for doc in X_train]
X_test_vec = [model_svc.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf_svc.fit(X_train_vec, y_train)
y_pred = clf_svc.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.77      0.83      0.80        24
           1       0.81      0.74      0.77        23

    accuracy                           0.79        47
   macro avg       0.79      0.79      0.79        47
weighted avg       0.79      0.79      0.79        47



In [13]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300)
    window = trial.suggest_int('window', 2, 10)
    min_count = trial.suggest_int('min_count', 1, 10)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    negative = trial.suggest_int('negative', 3, 15)
    dm = trial.suggest_categorical('dm', [0, 1])

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, negative=negative, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # XGBOOST 하이퍼파라미터 튜닝
    max_depth = trial.suggest_int('max_depth', 2, 30)
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    gamma = trial.suggest_float('gamma', 0, 1)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 20)
    
    # XGboost 모델 생성
    model = XGBClassifier(max_depth=max_depth,
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        gamma=gamma,
                        min_child_weight=min_child_weight,
                        random_state=123)
    
    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-21 15:11:50,518] A new study created in memory with name: no-name-9c084b2d-b8db-42d3-9e57-fe69af385a77
[I 2024-06-21 15:12:02,594] Trial 2 finished with value: 0.5964912280701754 and parameters: {'vector_size': 194, 'window': 7, 'min_count': 5, 'epochs': 30, 'negative': 3, 'dm': 0, 'max_depth': 29, 'n_estimators': 398, 'learning_rate': 0.25898551721887775, 'gamma': 0.2881348706436023, 'min_child_weight': 13}. Best is trial 2 with value: 0.5964912280701754.
[I 2024-06-21 15:12:05,295] Trial 1 finished with value: 0.6666666666666666 and parameters: {'vector_size': 50, 'window': 4, 'min_count': 9, 'epochs': 20, 'negative': 5, 'dm': 1, 'max_depth': 29, 'n_estimators': 286, 'learning_rate': 0.14055819859256774, 'gamma': 0.3483642813788431, 'min_child_weight': 13}. Best is trial 1 with value: 0.6666666666666666.
[I 2024-06-21 15:12:11,082] Trial 3 finished with value: 0.5581395348837209 and parameters: {'vector_size': 145, 'window': 8, 'min_count': 2, 'epochs': 25, 'negative': 4, 

Best hyperparameters:  {'vector_size': 198, 'window': 2, 'min_count': 4, 'epochs': 20, 'negative': 15, 'dm': 0, 'max_depth': 5, 'n_estimators': 306, 'learning_rate': 0.24930290167798524, 'gamma': 0.38536205788733313, 'min_child_weight': 7}


In [16]:
best_params = study.best_params

# Doc2Vec 모델 생성
model_xgb = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], 
                negative=best_params['negative'], seed=123)

# XGBoost 모델 생성
clf_xgb = XGBClassifier(
    max_depth=best_params['max_depth'],
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    gamma=best_params['gamma'],
    min_child_weight=best_params['min_child_weight'],
    random_state=123
)

# Doc2Vec 벡터 생성
X_train_vec = [model_xgb.infer_vector(doc) for doc in X_train]
X_test_vec = [model_xgb.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf_xgb.fit(X_train_vec, y_train)
y_pred = clf_xgb.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.67      0.75      0.71        24
           1       0.70      0.61      0.65        23

    accuracy                           0.68        47
   macro avg       0.68      0.68      0.68        47
weighted avg       0.68      0.68      0.68        47



In [17]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300)
    window = trial.suggest_int('window', 2, 10)
    min_count = trial.suggest_int('min_count', 1, 10)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    negative = trial.suggest_int('negative', 3, 15)
    dm = trial.suggest_categorical('dm', [0, 1])

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, negative=negative, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # LightGBM 하이퍼파라미터 튜닝
    max_depth = trial.suggest_int('max_depth', 2, 30)
    num_leaves = trial.suggest_int('num_leaves', 10, 200)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    min_child_samples = trial.suggest_int('min_child_samples', 3, 50)

    # LightGBM 모델 생성
    model = LGBMClassifier(
        max_depth=max_depth,
        num_leaves=num_leaves,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        min_child_samples=min_child_samples,
        random_state=123
    )
    
    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-21 15:20:54,393] A new study created in memory with name: no-name-64ecc5a3-a92e-4561-8ca2-91d4baab70b2
[I 2024-06-21 15:21:07,531] Trial 1 finished with value: 0.6122448979591837 and parameters: {'vector_size': 97, 'window': 5, 'min_count': 10, 'epochs': 30, 'negative': 7, 'dm': 0, 'max_depth': 2, 'num_leaves': 155, 'learning_rate': 0.2340357081790742, 'n_estimators': 82, 'min_child_samples': 26}. Best is trial 1 with value: 0.6122448979591837.
[I 2024-06-21 15:21:08,289] Trial 5 finished with value: 0.4444444444444444 and parameters: {'vector_size': 282, 'window': 5, 'min_count': 1, 'epochs': 30, 'negative': 3, 'dm': 0, 'max_depth': 10, 'num_leaves': 28, 'learning_rate': 0.22741857022844272, 'n_estimators': 85, 'min_child_samples': 25}. Best is trial 1 with value: 0.6122448979591837.
[I 2024-06-21 15:21:11,186] Trial 7 finished with value: 0.6 and parameters: {'vector_size': 163, 'window': 10, 'min_count': 5, 'epochs': 35, 'negative': 6, 'dm': 0, 'max_depth': 18, 'num_leave

Best hyperparameters:  {'vector_size': 248, 'window': 10, 'min_count': 8, 'epochs': 50, 'negative': 5, 'dm': 0, 'max_depth': 20, 'num_leaves': 151, 'learning_rate': 0.1905387071611366, 'n_estimators': 228, 'min_child_samples': 23}


In [18]:
best_params = study.best_params

# Doc2Vec 모델 생성
model_lgbm = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], 
                negative=best_params['negative'], seed=123)

# LightGBM 모델 생성
clf_lgbm = LGBMClassifier(max_depth=best_params['max_depth'], num_leaves=best_params['num_leaves'], 
                    learning_rate=best_params['learning_rate'], n_estimators=best_params['n_estimators'], 
                    min_child_samples=best_params['min_child_samples'], random_state=123)


# Doc2Vec 벡터 생성
X_train_vec = [model_lgbm.infer_vector(doc) for doc in X_train]
X_test_vec = [model_lgbm.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf_lgbm.fit(X_train_vec, y_train)
y_pred = clf_lgbm.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

[LightGBM] [Info] Number of positive: 54, number of negative: 53
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000577 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9265
[LightGBM] [Info] Number of data points in the train set: 107, number of used features: 248
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504673 -> initscore=0.018692
[LightGBM] [Info] Start training from score 0.018692
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

In [19]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300)
    window = trial.suggest_int('window', 2, 10)
    min_count = trial.suggest_int('min_count', 1, 10)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    negative = trial.suggest_int('negative', 3, 15)
    dm = trial.suggest_categorical('dm', [0, 1])

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, negative=negative, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # Logistic Regression 하이퍼파라미터 튜닝
    C = trial.suggest_float('C', 0.01, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])

    # Logistic Regression 모델 생성
    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver=solver,
        random_state=123
    )
    
    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-21 15:31:12,405] A new study created in memory with name: no-name-3843ce9f-7a46-4d46-af26-9458aa6d2a2c
[I 2024-06-21 15:31:26,681] Trial 3 finished with value: 0.6571428571428571 and parameters: {'vector_size': 292, 'window': 8, 'min_count': 4, 'epochs': 30, 'negative': 6, 'dm': 0, 'C': 0.047588688040548534, 'penalty': 'l1', 'solver': 'saga'}. Best is trial 3 with value: 0.6571428571428571.
[I 2024-06-21 15:31:26,982] Trial 6 finished with value: 0.5833333333333334 and parameters: {'vector_size': 129, 'window': 3, 'min_count': 9, 'epochs': 35, 'negative': 8, 'dm': 0, 'C': 2.679958099099637, 'penalty': 'l1', 'solver': 'liblinear'}. Best is trial 3 with value: 0.6571428571428571.
[I 2024-06-21 15:31:28,916] Trial 5 finished with value: 0.0 and parameters: {'vector_size': 74, 'window': 6, 'min_count': 6, 'epochs': 40, 'negative': 8, 'dm': 0, 'C': 0.012489286185177759, 'penalty': 'l1', 'solver': 'saga'}. Best is trial 3 with value: 0.6571428571428571.
[I 2024-06-21 15:31:31,185]

Best hyperparameters:  {'vector_size': 216, 'window': 6, 'min_count': 10, 'epochs': 40, 'negative': 14, 'dm': 0, 'C': 0.027532020293179145, 'penalty': 'l2', 'solver': 'liblinear'}


In [21]:
best_params = study.best_params

# Doc2Vec 모델 생성
model_lr = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], 
                negative=best_params['negative'], seed=123)

# Logistic Regressor 모델 생성
clf_lr = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver=best_params['solver'], random_state=123)

# Doc2Vec 벡터 생성
X_train_vec = [model_lr.infer_vector(doc) for doc in X_train]
X_test_vec = [model_lr.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf_lr.fit(X_train_vec, y_train)
y_pred = clf_lr.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.76      0.54      0.63        24
           1       0.63      0.83      0.72        23

    accuracy                           0.68        47
   macro avg       0.70      0.68      0.68        47
weighted avg       0.70      0.68      0.67        47



In [22]:
file_path = r'.\crawling_data\preprocessed_essay.csv'

# 자소서 리스트화
documents = []
with open(file_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        documents.append(row[0])

# 사용하지 않은 2점짜리 자소서 리스트
documents = documents[-175:-77]

In [23]:
# 불용어 리스트 가져오기
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

documents_corpus = []
for document in documents:
    documents_corpus.append(preprocess(document))

In [25]:
rf_vec = [model_rf.infer_vector(doc) for doc in documents_corpus]
svc_vec = [model_svc.infer_vector(doc) for doc in documents_corpus]
xgb_vec = [model_xgb.infer_vector(doc) for doc in documents_corpus]
lgbm_vec = [model_lgbm.infer_vector(doc) for doc in documents_corpus]
lr_vec = [model_lr.infer_vector(doc) for doc in documents_corpus]

print(clf_rf.predict(rf_vec).sum())
print(clf_svc.predict(svc_vec).sum())
print(clf_xgb.predict(xgb_vec).sum())
print(clf_lgbm.predict(lgbm_vec).sum())
print(clf_lr.predict(lr_vec).sum())

59
37
48
30
68


In [26]:
import pickle

with open('d2v_model_rf.pkl', 'wb') as f:
    pickle.dump(model_rf, f)

with open('d2v_model_svc.pkl', 'wb') as f:
    pickle.dump(model_svc, f)

with open('d2v_model_xgb.pkl', 'wb') as f:
    pickle.dump(model_xgb, f)

with open('d2v_model_lgbm.pkl', 'wb') as f:
    pickle.dump(model_lgbm, f)

with open('d2v_model_lr.pkl', 'wb') as f:
    pickle.dump(model_lr, f)
    
with open('clf_model_rf.pkl', 'wb') as f:
    pickle.dump(clf_rf, f)

with open('clf_model_svc.pkl', 'wb') as f:
    pickle.dump(clf_svc, f)
    
with open('clf_model_xgb.pkl', 'wb') as f:
    pickle.dump(clf_xgb, f)

with open('clf_model_lgbm.pkl', 'wb') as f:
    pickle.dump(clf_lgbm, f)

with open('clf_model_lr.pkl', 'wb') as f:
    pickle.dump(clf_lr, f)